<a href="https://colab.research.google.com/github/kiat/Cloud-Computing/blob/main/Spark-Example-TPCH/Colab_TPC_H_Example_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# !tar xf spark-3.0.1-bin-hadoop3.2.tgz
# !pip install -q findspark

!pip install --ignore-installed -q pyspark==3.0.1

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 


In [2]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext


import sys
from operator import add

from pyspark import SparkContext
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
import sys
import requests
from operator import add

from pyspark.sql.types import *
from pyspark.sql import functions as func

from pyspark.sql.functions import lit
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.functions import array

from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

# conf = SparkConf(appName="TPCH-Example")
# create spark context with the above configuration
# sc = SparkContext(conf=conf)

sqlContext = SQLContext(sc)


In [3]:
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/customer.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/lineitem.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/nation.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/orders.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/region.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/supplier.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/part.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/region.tbl
! wget -q https://raw.githubusercontent.com/kiat/Cloud-Computing/main/Spark-Example-TPCH/tpch_tables_scale_0.1/partsupp.tbl

In [4]:
! ls -la 

total 105340
drwxr-xr-x 1 root root     4096 Mar 24 16:16 .
drwxr-xr-x 1 root root     4096 Mar 24 16:15 ..
drwxr-xr-x 1 root root     4096 Mar  9 14:47 .config
-rw-r--r-- 1 root root  2426178 Mar 24 16:16 customer.tbl
-rw-r--r-- 1 root root 74247152 Mar 24 16:16 lineitem.tbl
-rw-r--r-- 1 root root     2257 Mar 24 16:16 nation.tbl
-rw-r--r-- 1 root root 16893215 Mar 24 16:16 orders.tbl
-rw-r--r-- 1 root root 11728237 Mar 24 16:16 partsupp.tbl
-rw-r--r-- 1 root root  2391154 Mar 24 16:16 part.tbl
-rw-r--r-- 1 root root      412 Mar 24 16:16 region.tbl
-rw-r--r-- 1 root root      412 Mar 24 16:16 region.tbl.1
drwxr-xr-x 1 root root     4096 Mar  9 14:48 sample_data
-rw-r--r-- 1 root root   139679 Mar 24 16:16 supplier.tbl


In [5]:
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("lineitem.tbl")
lineitems.show()

+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|ORDERKEY|PARTKEY|SUPPKEY|LINENUMBER|QUANTITY|EXTENDEDPRICE|DISCOUNT| TAX|RETURNFLAG|LINESTATUS|  SHIPDATE|COMMITDATE|RECEIPTDATE|     SHIPINSTRUCT|SHIPMODE|             COMMENT|
+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|       1|  15519|    785|         1|      17|     24386.67|    0.04|0.02|         N|         O|1996-03-13|1996-02-12| 1996-03-22|DELIVER IN PERSON|   TRUCK|egular courts abo...|
|       1|   6731|    732|         2|      36|     58958.28|    0.09|0.06|         N|         O|1996-04-12|1996-02-28| 1996-04-20| TAKE BACK RETURN|    MAIL|ly final dependen...|
|       1|   6370|    371|         3|       8|     10210.96|     0.1|0.02|         N|         O|1996-01-2

In [6]:

customer = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("customer.tbl")
orders = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("orders.tbl")
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("lineitem.tbl")
part = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("part.tbl")
supplier = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("supplier.tbl")
partsupp = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("partsupp.tbl")
region = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("region.tbl")
nation = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load("nation.tbl")



In [7]:
nation.show()

+---------+----------+---------+--------------------+
|NATIONKEY|      NAME|REGIONKEY|             COMMENT|
+---------+----------+---------+--------------------+
|        0|   ALGERIA|        0| haggle. carefull...|
|        1| ARGENTINA|        1|al foxes promise ...|
|        2|    BRAZIL|        1|y alongside of th...|
|        3|    CANADA|        1|eas hang ironic, ...|
|        4|     EGYPT|        4|y above the caref...|
|        5|  ETHIOPIA|        0|ven packages wake...|
|        6|    FRANCE|        3|refully final req...|
|        7|   GERMANY|        3|l platelets. regu...|
|        8|     INDIA|        2|ss excuses cajole...|
|        9| INDONESIA|        2| slyly express as...|
|       10|      IRAN|        4|efully alongside ...|
|       11|      IRAQ|        4|nic deposits boos...|
|       12|     JAPAN|        2|ously. final, exp...|
|       13|    JORDAN|        4|ic deposits are b...|
|       14|     KENYA|        0| pending excuses ...|
|       15|   MOROCCO|      